- # Full fine-tuning

### 1. Select LLM

In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
model = T5ForConditionalGeneration.from_pretrained('google-t5/t5-small')

d:\PersonalWork\Learning_GenAI\fine-tuning\fine-tuning-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. Select Tokenizer

In [2]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


### 3. Choose & Load the Dataset

In [3]:
from datasets import load_dataset
# Load a dataset (this example uses a simple summarization dataset)
dataset = load_dataset("cnn_dailymail", "3.0.0", split='train[:1%]')  # Only using 1% for low-resource setting

### 4. Tokenize the input

In [4]:
def preprocess_function(examples):
    inputs = ["summerize: " + doc for doc in examples['article']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length", return_tensors="pt")
    labels = tokenizer(examples['highlights'], max_length=150, truncation=True, padding="max_length", return_tensors="pt").input_ids
    model_inputs["labels"] = labels
    return model_inputs

### 5. Tokenize the dataset

In [5]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

### 6. Define training arguments

In [6]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Set evaluation strategy to 'epoch' or 'steps'
    save_strategy="epoch",
    per_device_train_batch_size=6,
    num_train_epochs=3,
    eval_steps=100,
    save_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,  # Required when using EarlyStoppingCallback
)

d:\PersonalWork\Learning_GenAI\fine-tuning\fine-tuning-env\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### 7. Initialize the Trainer

In [7]:
from transformers import Trainer, EarlyStoppingCallback
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset=tokenized_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],  # Stops after 3 epochs without improvement
)

### 8. Train the model

In [8]:
trainer.train()

  0%|          | 2/1437 [00:26<5:16:14, 13.22s/it]

KeyboardInterrupt: 